# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [13]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    int_to_vocab = {i : word for i, word in enumerate(vocab)}
    vocab_to_int = {word : i for i, word in enumerate(vocab)}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [16]:
# Extra test code
test_text = ['moe_szyslak', "moe's", 'tavern', 'where', 'the', 'elite', 'meet', 'to', 'drink', 'bart_simpson', 'eh', 'yeah', 'hello', 'is', 'mike', 'there', 'last', 'name', 'rotch', 'moe_szyslak', 'hold', 'on', "i'll", 'check', 'mike', 'rotch', 'mike', 'rotch', 'hey', 'has', 'anybody', 'seen', 'mike', 'rotch', 'lately', 'moe_szyslak', 'listen', 'you', 'little', 'puke', 'one', 'of', 'these', 'days', "i'm", 'gonna', 'catch', 'you', 'and', "i'm", 'gonna', 'carve', 'my', 'name', 'on', 'your', 'back', 'with', 'an', 'ice', 'pick', 'moe_szyslak', 'whats', 'the', 'matter', 'homer', "you're", 'not', 'your', 'normal', 'effervescent', 'self', 'homer_simpson', 'i', 'got', 'my', 'problems', 'moe', 'give', 'me', 'another', 'one', 'moe_szyslak', 'homer', 'hey', 'you', 'should', 'not', 'drink', 'to', 'forget', 'your', 'problems', 'barney_gumble', 'yeah', 'you', 'should', 'only', 'drink', 'to', 'enhance', 'your', 'social', 'skills']
#print(len(test_text), len(set(test_text)))
v_to_i, i_to_v = create_lookup_tables(test_text)
print(v_to_i, len(v_to_i))
print(i_to_v, len(i_to_v))

{'tavern': 0, 'with': 1, 'and': 2, 'you': 3, 'puke': 4, 'problems': 5, 'last': 6, 'gonna': 7, 'me': 8, 'yeah': 9, 'drink': 10, 'anybody': 11, 'seen': 12, 'moe_szyslak': 13, 'meet': 14, 'lately': 15, 'the': 16, 'social': 17, 'matter': 18, 'catch': 19, 'hello': 20, 'forget': 21, 'got': 22, 'where': 23, 'these': 24, 'on': 25, 'carve': 26, 'of': 27, "you're": 28, 'self': 29, 'ice': 30, 'your': 31, 'name': 32, 'barney_gumble': 33, 'there': 34, 'check': 35, 'moe': 36, 'enhance': 37, 'days': 38, 'back': 39, 'homer_simpson': 40, 'not': 41, 'should': 42, 'give': 43, 'only': 44, "i'll": 45, 'hey': 46, 'elite': 47, 'hold': 48, 'has': 49, "i'm": 50, 'rotch': 51, 'mike': 52, 'an': 53, 'one': 54, 'whats': 55, 'normal': 56, 'i': 57, 'to': 58, 'effervescent': 59, 'skills': 60, 'eh': 61, 'another': 62, 'homer': 63, 'my': 64, 'is': 65, "moe's": 66, 'listen': 67, 'bart_simpson': 68, 'pick': 69, 'little': 70} 71
{0: 'tavern', 1: 'with', 2: 'and', 3: 'you', 4: 'puke', 5: 'problems', 6: 'last', 7: 'gonna', 

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [19]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    tokenize_dict = { \
        '.' : '||Period||',
        ',' : '||Comma||',
        '\"': '||Quotation_Mark||',
        ';' : '||Semicolon||',
        '!' : '||Exclamation_mark||',
        '?' : '||Question_mark||',
        '(' : '||Left_Parentheses||',
        ')' : '||Right_Parentheses||',
        '--': '||Dash||',
        '\n': '||Return||'}
    return tokenize_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

NameError: name 'token_lookup' is not defined

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.0


C:\Anaconda3\envs\tv-script-generation\lib\site-packages\ipykernel_launcher.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.
  


In [4]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[ 22.  28.]
 [ 49.  64.]]


In [7]:
# Creates a graph.
#with tf.device('/cpu:0'):
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'b_4': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/cpu:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: b_4 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [3,2] values: [1 2][3]...>, _device="/device:GPU:0"]()]]

Caused by op 'b_4', defined at:
  File "C:\Anaconda3\envs\tv-script-generation\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\envs\tv-script-generation\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Anaconda3\envs\tv-script-generation\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Anaconda3\envs\tv-script-generation\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Anaconda3\envs\tv-script-generation\lib\site-packages\ipykernel\k

### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [4]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], 'input')
    targets = tf.placeholder(tf.int32, [None, None], 'targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return input, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [5]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    num_layers = 2
    lstm = tf.contrib.rnn.BasicLSTMCell
    cell = tf.contrib.rnn.MultiRNNCell([lstm(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, 'initial_state')
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [6]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform(shape=[vocab_size, embed_dim],
                                              minval=-1,
                                              maxval=1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [7]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [8]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    rnn, final_state = build_rnn(cell, embed)
    logits = tf.contrib.layers.fully_connected(rnn, vocab_size, activation_fn=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [9]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    characters_per_batch = batch_size * seq_length
    n_batches = len(int_text) // characters_per_batch
    batches = np.zeros((n_batches, 2, batch_size, seq_length),
                       dtype=np.int32)
    text = int_text[: n_batches*characters_per_batch]
    text = np.array(text)
    text = text.reshape((batch_size, -1))
    
    last_col = []
    total_seq_length = n_batches * seq_length
    for n in range(total_seq_length, text.size, total_seq_length):
        last_col.append(int_text[n])
    last_col.append(int_text[0])
    last_col = np.array(last_col, ndmin=2)
    
    text = np.concatenate((text, last_col.T), axis=1)
    
    for n in range(0, n_batches):
        input = text[:, seq_length*n : seq_length*(n+1)]
        batches[n, 0] = input
        target = text[:, seq_length*n+1 : seq_length*(n+1)+1]
        batches[n, 1] = target
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


In [10]:
get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 3, 2)

array([[[[ 1,  2],
         [ 5,  6],
         [ 9, 10]],

        [[ 2,  3],
         [ 6,  7],
         [10, 11]]],


       [[[ 3,  4],
         [ 7,  8],
         [11, 12]],

        [[ 4,  5],
         [ 8,  9],
         [12,  1]]]])

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [11]:
# Number of Epochs
num_epochs = 500 #100
# Batch Size
batch_size = 64 #32 ###10
# RNN Size
rnn_size = 128 ###128
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 50 ###50
# Learning Rate
learning_rate = .01 ###.01
# Show stats for every n number of batches
show_every_n_batches = 10

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.821
Epoch   0 Batch   10/21   train_loss = 6.693
Epoch   0 Batch   20/21   train_loss = 6.666
Epoch   1 Batch    9/21   train_loss = 6.057
Epoch   1 Batch   19/21   train_loss = 6.119
Epoch   2 Batch    8/21   train_loss = 6.056
Epoch   2 Batch   18/21   train_loss = 6.135
Epoch   3 Batch    7/21   train_loss = 6.010
Epoch   3 Batch   17/21   train_loss = 6.012
Epoch   4 Batch    6/21   train_loss = 6.071
Epoch   4 Batch   16/21   train_loss = 5.853
Epoch   5 Batch    5/21   train_loss = 5.781
Epoch   5 Batch   15/21   train_loss = 5.766
Epoch   6 Batch    4/21   train_loss = 5.573
Epoch   6 Batch   14/21   train_loss = 5.456
Epoch   7 Batch    3/21   train_loss = 5.401
Epoch   7 Batch   13/21   train_loss = 5.253
Epoch   8 Batch    2/21   train_loss = 5.340
Epoch   8 Batch   12/21   train_loss = 5.363
Epoch   9 Batch    1/21   train_loss = 5.185
Epoch   9 Batch   11/21   train_loss = 5.219
Epoch  10 Batch    0/21   train_loss = 5.215
Epoch  10 

Epoch  87 Batch    3/21   train_loss = 1.173
Epoch  87 Batch   13/21   train_loss = 1.201
Epoch  88 Batch    2/21   train_loss = 1.265
Epoch  88 Batch   12/21   train_loss = 1.111
Epoch  89 Batch    1/21   train_loss = 1.229
Epoch  89 Batch   11/21   train_loss = 1.163
Epoch  90 Batch    0/21   train_loss = 1.191
Epoch  90 Batch   10/21   train_loss = 1.130
Epoch  90 Batch   20/21   train_loss = 1.192
Epoch  91 Batch    9/21   train_loss = 1.145
Epoch  91 Batch   19/21   train_loss = 1.161
Epoch  92 Batch    8/21   train_loss = 1.125
Epoch  92 Batch   18/21   train_loss = 1.125
Epoch  93 Batch    7/21   train_loss = 1.089
Epoch  93 Batch   17/21   train_loss = 1.161
Epoch  94 Batch    6/21   train_loss = 1.100
Epoch  94 Batch   16/21   train_loss = 1.180
Epoch  95 Batch    5/21   train_loss = 1.105
Epoch  95 Batch   15/21   train_loss = 1.086
Epoch  96 Batch    4/21   train_loss = 1.119
Epoch  96 Batch   14/21   train_loss = 1.117
Epoch  97 Batch    3/21   train_loss = 1.059
Epoch  97 

Epoch 174 Batch    6/21   train_loss = 0.646
Epoch 174 Batch   16/21   train_loss = 0.684
Epoch 175 Batch    5/21   train_loss = 0.642
Epoch 175 Batch   15/21   train_loss = 0.609
Epoch 176 Batch    4/21   train_loss = 0.692
Epoch 176 Batch   14/21   train_loss = 0.672
Epoch 177 Batch    3/21   train_loss = 0.646
Epoch 177 Batch   13/21   train_loss = 0.652
Epoch 178 Batch    2/21   train_loss = 0.652
Epoch 178 Batch   12/21   train_loss = 0.588
Epoch 179 Batch    1/21   train_loss = 0.671
Epoch 179 Batch   11/21   train_loss = 0.640
Epoch 180 Batch    0/21   train_loss = 0.622
Epoch 180 Batch   10/21   train_loss = 0.581
Epoch 180 Batch   20/21   train_loss = 0.616
Epoch 181 Batch    9/21   train_loss = 0.615
Epoch 181 Batch   19/21   train_loss = 0.621
Epoch 182 Batch    8/21   train_loss = 0.594
Epoch 182 Batch   18/21   train_loss = 0.542
Epoch 183 Batch    7/21   train_loss = 0.516
Epoch 183 Batch   17/21   train_loss = 0.556
Epoch 184 Batch    6/21   train_loss = 0.504
Epoch 184 

Epoch 261 Batch    9/21   train_loss = 0.267
Epoch 261 Batch   19/21   train_loss = 0.298
Epoch 262 Batch    8/21   train_loss = 0.275
Epoch 262 Batch   18/21   train_loss = 0.250
Epoch 263 Batch    7/21   train_loss = 0.247
Epoch 263 Batch   17/21   train_loss = 0.262
Epoch 264 Batch    6/21   train_loss = 0.252
Epoch 264 Batch   16/21   train_loss = 0.277
Epoch 265 Batch    5/21   train_loss = 0.265
Epoch 265 Batch   15/21   train_loss = 0.232
Epoch 266 Batch    4/21   train_loss = 0.276
Epoch 266 Batch   14/21   train_loss = 0.257
Epoch 267 Batch    3/21   train_loss = 0.248
Epoch 267 Batch   13/21   train_loss = 0.255
Epoch 268 Batch    2/21   train_loss = 0.274
Epoch 268 Batch   12/21   train_loss = 0.249
Epoch 269 Batch    1/21   train_loss = 0.287
Epoch 269 Batch   11/21   train_loss = 0.276
Epoch 270 Batch    0/21   train_loss = 0.285
Epoch 270 Batch   10/21   train_loss = 0.241
Epoch 270 Batch   20/21   train_loss = 0.273
Epoch 271 Batch    9/21   train_loss = 0.260
Epoch 271 

Epoch 348 Batch   12/21   train_loss = 0.233
Epoch 349 Batch    1/21   train_loss = 0.292
Epoch 349 Batch   11/21   train_loss = 0.289
Epoch 350 Batch    0/21   train_loss = 0.288
Epoch 350 Batch   10/21   train_loss = 0.258
Epoch 350 Batch   20/21   train_loss = 0.275
Epoch 351 Batch    9/21   train_loss = 0.248
Epoch 351 Batch   19/21   train_loss = 0.279
Epoch 352 Batch    8/21   train_loss = 0.284
Epoch 352 Batch   18/21   train_loss = 0.269
Epoch 353 Batch    7/21   train_loss = 0.248
Epoch 353 Batch   17/21   train_loss = 0.273
Epoch 354 Batch    6/21   train_loss = 0.274
Epoch 354 Batch   16/21   train_loss = 0.294
Epoch 355 Batch    5/21   train_loss = 0.282
Epoch 355 Batch   15/21   train_loss = 0.294
Epoch 356 Batch    4/21   train_loss = 0.314
Epoch 356 Batch   14/21   train_loss = 0.301
Epoch 357 Batch    3/21   train_loss = 0.312
Epoch 357 Batch   13/21   train_loss = 0.326
Epoch 358 Batch    2/21   train_loss = 0.320
Epoch 358 Batch   12/21   train_loss = 0.315
Epoch 359 

Epoch 435 Batch   15/21   train_loss = 0.872
Epoch 436 Batch    4/21   train_loss = 0.931
Epoch 436 Batch   14/21   train_loss = 0.922
Epoch 437 Batch    3/21   train_loss = 0.896
Epoch 437 Batch   13/21   train_loss = 1.019
Epoch 438 Batch    2/21   train_loss = 0.982
Epoch 438 Batch   12/21   train_loss = 0.938
Epoch 439 Batch    1/21   train_loss = 1.024
Epoch 439 Batch   11/21   train_loss = 1.000
Epoch 440 Batch    0/21   train_loss = 0.936
Epoch 440 Batch   10/21   train_loss = 1.032
Epoch 440 Batch   20/21   train_loss = 0.997
Epoch 441 Batch    9/21   train_loss = 0.900
Epoch 441 Batch   19/21   train_loss = 0.910
Epoch 442 Batch    8/21   train_loss = 0.828
Epoch 442 Batch   18/21   train_loss = 0.750
Epoch 443 Batch    7/21   train_loss = 0.746
Epoch 443 Batch   17/21   train_loss = 0.738
Epoch 444 Batch    6/21   train_loss = 0.743
Epoch 444 Batch   16/21   train_loss = 0.721
Epoch 445 Batch    5/21   train_loss = 0.628
Epoch 445 Batch   15/21   train_loss = 0.591
Epoch 446 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [14]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    input = loaded_graph.get_tensor_by_name(name="input:0")
    initial_state = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state = loaded_graph.get_tensor_by_name("final_state:0")
    probs = loaded_graph.get_tensor_by_name("probs:0")        
    return input, initial_state, final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [15]:
import operator
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    NUM_OF_MOST_PROB_WORDS = 3
    def max_prob():
        i = max(probs_d, key=lambda key: probs_d[key])
        p = float(probs_d[i])
        del probs_d[i]
        yield i,p
    
    probs_d = {i:p for i,p in enumerate(probabilities)}
    num_of_most_prob = len(probabilities) if NUM_OF_MOST_PROB_WORDS > len(probabilities) \
                                          else NUM_OF_MOST_PROB_WORDS
    most_probs = {}
    for _ in range(num_of_most_prob):
        i,p = next(max_prob())
        most_probs[i] = p
    most_probs_values = np.array(list(most_probs.values()))
    most_probs_values = most_probs_values / (most_probs_values.sum())
    predicted_word_int = np.random.choice(list(most_probs.keys()),
                                          p=most_probs_values)
    predicted_word = int_to_vocab[predicted_word_int]
    return predicted_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [20]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        ### pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)
        ### This lead to TypeError: only length-1 arrays can be converted to Python scalars
        ### tensorflow lower then 1.2.0 unavaliable via pip
        ### Using this line of code
        pred_word = pick_word(probabilities[0,dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: sorry, homer. how one beer, you...
moe_szyslak: who's donatin' his body for that one, there was jack all the other way.
carl_carlson: homer, that is a ticket to see" / that's homer?
moe_szyslak: i don't get this!
marge_simpson: i got a very new self. like actually say, i'm in the ball.
moe_szyslak: there he is.
attractive_woman_#1: let's never own beer, the next date with me.
lenny_leonard: to the best thing better than it!
moe_szyslak:(surprised) my bar all day they say this beers will have a feeling? i'm dutch. eh, forget you you have a little dinner next from the family-owned teams.
moe_szyslak: hey, homer, can one get it.
young_homer: oh, i'm a little busy, this is, who am i am very?
homer_simpson: what the princesses was a world, here... this thing's just got a girls spits failed a place named boxcar bob, ones, duff is a game of speech i


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.